In [5]:
# %%
from transformers import pipeline, set_seed
from datasets import load_dataset
import pandas as pd
import json
import regex as re
import nltk
from nltk.tokenize import sent_tokenize
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import ast
import torch
from huggingface_hub import scan_cache_dir
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoConfig
import ast
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
import os
from datasets import DatasetDict, concatenate_datasets
from sentence_transformers import SentenceTransformer
from joblib import dump, load
import argparse
import joblib

#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

#watch -n0.1 nvidia-smi

In [2]:
### Load model

cuda_count = torch.cuda.device_count()

sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2').to(device)

# model_name  = "Open-Orca/Mistral-7B-OpenOrca" #"mistralai/Mistral-7B-v0.1
# config = AutoConfig.from_pretrained(model_name)
# max_input_size =  config.max_position_embeddings  
# tokenizer = AutoTokenizer.from_pretrained(model_name, device=device, )


In [3]:
def getIdsType(brief_type):
    def getIds(briefs):

        briefs = briefs[brief_type]
        briefs_ids = []
        briefs = ast.literal_eval(briefs)
        for brief in briefs:
            iD = tokenizer(brief, max_length = max_input_size, padding='max_length', truncation= True ,return_tensors="pt") #.to(device).input_ids 
            briefs_ids.append(iD)
        return { f"ids_{brief_type}": briefs_ids }
    return getIds


def meanEmbeddings(briefs):

    briefs = ast.literal_eval(briefs)
    # Place model inputs on the GPU
    embeddings = []
    for brief in briefs:
        argument = tokenizer(brief, max_length = max_input_size , padding="max_length" ,truncation= True ,return_tensors="pt").to(device) 
        # Extract last hidden states
        model.eval()
        with torch.no_grad():
            argument = argument.to(device)
            output = model(**argument)
            argument = argument.to(device)
            last_hidden_state = output.last_hidden_state


        last_hidden_state = last_hidden_state.reshape(( max_input_size, config.hidden_size))
        mask = argument['attention_mask'].to(device).bool()
        mask = mask.reshape(max_input_size)
        mask = mask.nonzero().squeeze()
        hidden_states = torch.index_select(last_hidden_state, 0, mask)
        
        #print(last_hidden_state)
        inputs = get_mean_embedding(hidden_states.cpu().to(torch.float64).numpy()).tolist()
        del last_hidden_state
        del output
        del mask
        del hidden_states
        del argument
        
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
                
        embeddings.append(inputs)
    # Return vector for [CLS] token
    return embeddings

def summerizeEmbeddings(brief):
    embedding = sentence_model.encode(brief).tolist()
    return embedding



def get_mean_embedding(embedding):
    return np.mean(embedding, axis=0)


In [28]:
model_name = "KNN"
feature = 'tfidf'
#for model_name in ["RFT", "SGD"]: #["KNN","LinearSVC", "Logistic", "RFT", "SGD"]:
data = 'dataset/testset.csv'
if True: #__name__ == "__main__":
    model_types = ["KNN","LinearSVC", "Logistic", "RFT", "SGD"]
    # parser = argparse.ArgumentParser(description='End to end pipeline from briefs to predictions')
    # parser.add_argument('--model_name', default='RFT', help=f'There are {len(model_types)} models to choose from: {model_types}')
    # parser.add_argument('--data',  default= 'embeddings.csv')
    # parser.add_argument('--feature', default='sentence_embeddings', help='There are two features to choose from: sentence_embeddings and tfidf')



    # arg = parser.parse_args()

    testset = pd.read_csv(data, index_col=0)#pd.read_csv(arg.data, index_col=0)

    testset["completion"] = list(map(lambda x : x.strip() ,testset["completion"].to_list()))
    

    summerize = True

    if feature == 'sentence_embeddings':#if arg.feature == 'sentence_embeddings':
        if 'embeddings' not in testset.columns:
            if not summerize:
                model  = "mistralai/Mistral-7B-v0.1"
                config = AutoConfig.from_pretrained(model)
                max_input_size =  config.max_position_embeddings  
                tokenizer = AutoTokenizer.from_pretrained(model, device=device, )

                tokenizer.pad_token = tokenizer.eos_token
                model = AutoModel.from_pretrained(model, torch_dtype= torch.bfloat16 ).to(device)  #,device_map = "auto"
                #parallel_model = torch.nn.DataParallel(model)
            else:
                sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2').to(device)

            testset['feature'] = ""
            testset['feature'] = testset['prompt'].map(summerizeEmbeddings)
            testset.to_csv('embeddings.csv')


        else:
            #get model for predictions
            turn2list = lambda x: ast.literal_eval(x)
            testset['feature'] = testset['embeddings'].map(turn2list)


    elif feature == 'tfidf':# elif arg.feature == 'tfidf':
        print("Loading tfidf pipes")
        support_pipe = joblib.load('pipes/support-tfidf.joblib')
        opposition_pipe = joblib.load('pipes/oppose-tfidf.joblib')

        testset['feature'] = ""

        sparse_matrix = support_pipe.transform(testset["prompt"].loc[testset["brief_type"]=="support"]).toarray()

        testset['feature'].loc[testset["brief_type"]=="support"] =   sparse_matrix.tolist()

        sparse_matrix = opposition_pipe.transform(testset["prompt"].loc[testset["brief_type"]=="opposition"]).toarray()

        testset['feature'].loc[testset["brief_type"]=="opposition"] =   sparse_matrix.tolist()
        


    x_support = np.array(testset["feature"].loc[(testset["brief_type"]=="support") & (testset["data_type"]=="test") ].to_list())  #  np.array(testset['file_path'].to_list())
    x_opposition = np.array(testset["feature"].loc[(testset["brief_type"]=="opposition") & (testset["data_type"]=="test") ].to_list())  #  np.array(testset['file_path'].to_list())
        

    #get model for predictions

    support_model_path = f'models/{model_name}-support-{feature}.pkl'
    opposition_model_path = f'models/{model_name}-opposition-{feature}.pkl'

    clfs = {"sup" : load(support_model_path)  , "opp" : load(opposition_model_path) }

    if hasattr(clfs["sup"], 'predict_proba') and callable(getattr(clfs["sup"], 'predict_proba')):
        scores_support = clfs['sup'].predict_proba(x_support)
        scores_opposition = clfs['opp'].predict_proba(x_opposition)
    else:
        scores_support = clfs['sup'].decision_function(x_support)
        scores_opposition = clfs['opp'].decision_function(x_opposition)

    prediction_opposition = clfs['opp'].predict(x_opposition)
    prediction_support = clfs['sup'].predict(x_support)

    testset.rename(columns={"file_name": "brief","completion": "truth"} , inplace = True  )

    support = testset.loc[(testset["brief_type"]=="support") & (testset["data_type"]=="test") ].copy()
    opposition = testset.loc[(testset["brief_type"]=="opposition") & (testset["data_type"]=="test") ].copy()

    support.drop(['data_type','prompt' , 'brief_type','file_path','feature'], axis=1, inplace=True)
    opposition.drop(['data_type', 'prompt' , 'brief_type','file_path','feature'], axis=1, inplace=True)

    support['predict'] = ""
    opposition['predict'] = ""

    support['predict'] = prediction_support
    opposition['predict'] = prediction_opposition

    support['score'] = ""
    opposition['score'] = ""

    support['score'] = list(map( np.max ,scores_support.tolist()))
    opposition['score']= list(map(  np.max ,scores_opposition.tolist() ))

    support = support[["brief","predict","score","truth"]]
    opposition = opposition[["brief","predict","score","truth"]]

    support.to_csv(f'{model_name}-{feature}-supppredictions.csv' , index = False)
    opposition.to_csv(f'{model_name}-{feature}-oppopredictions.csv', index = False)


Loading tfidf pipes


/var/folders/jd/_y9k6hnx4658pxlgs6r0d1pr0000gn/T/ipykernel_62893/1090808564.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testset['feature'].loc[testset["brief_type"]=="support"] =   sparse_matrix.tolist()
/var/folders/jd/_y9k6hnx4658pxlgs6r0d1pr0000gn/T/ipykernel_62893/1090808564.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testset['feature'].loc[testset["brief_type"]=="opposition"] =   sparse_matrix.tolist()


In [4]:
testset = pd.read_csv('dataset/testset.csv', index_col=0)
testset

,prompt,completion,brief_type,data_type,file_path,file_name
0,Case 9:11-cv-80416-KLR Document 737 Entered on...,deny,support,train,1013,gov.uscourts.flsd.377721.737.0.txt
1,Case 9:11-cv-80416-KLR Document 743 Entered on...,deny,opposition,train,1013,gov.uscourts.flsd.377721.743.0.txt
2,IN THE UNITED STATES DISTRICT COURT\nFOR THE S...,deny,support,train,909,gov.uscourts.mssd.95610.6.0.txt
3,"I i .\nt | | , : tl\n\n \n\n \n\n \n\n \n\n \n...",deny,opposition,train,909,gov.uscourts.mssd.95610.7.0.txt
4,Case 1:08-cv-11908-RWZ Document 27\n\nFiled 09...,grant,support,train,507,gov.uscourts.mad.118796.27.0.txt
...,...,...,...,...,...,...
1210,Case 6:10-cv-00111-LED Document 333\n\nFiled 0...,deny,opposition,test,621,gov.uscourts.txed.121829.333.0.txt
1211,"Helio LLC v. Palm, Inc. Doc. 3\n\n \n\n \n\n \...",deny,support,train,574,gov.uscourts.cand.187342.3.0.txt
1212,"Helio LLC v. Palm, lnc.\n\n60373/2021440.1\n\n...",deny,opposition,train,574,gov.uscourts.cand.187342.9.0.txt
1213,Case 1:11-cv-00696-RLW Document 4 Filed 04/08/...,grant,support,test,711,gov.uscourts.dcd.147524.4.0.txt


In [18]:
testset['tfidf'] = ""


sparse_matrix = pipe.transform(testset["prompt"].loc[testset["brief_type"]=="support"]).toarray()

testset['tfidf'].loc[testset["brief_type"]=="support"] =   sparse_matrix.tolist()

sparse_matrix = pipe.transform(testset["prompt"].loc[testset["brief_type"]=="opposition"]).toarray()

testset['tfidf'].loc[testset["brief_type"]=="opposition"] =   sparse_matrix.tolist()



/var/folders/jd/_y9k6hnx4658pxlgs6r0d1pr0000gn/T/ipykernel_62893/2777049630.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testset['tfidf'].loc[testset["brief_type"]=="support"] =   sparse_matrix.tolist()
/var/folders/jd/_y9k6hnx4658pxlgs6r0d1pr0000gn/T/ipykernel_62893/2777049630.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testset['tfidf'].loc[testset["brief_type"]=="opposition"] =   sparse_matrix.tolist()


In [23]:
arg = {"feature": "tfidf", "model_type": "KNN", "data": "embeddings.csv"}

arg.feature

AttributeError: 'dict' object has no attribute 'feature'

In [ ]:
# 1. run experiment with the last hidden state with just support
# TFI-DF and Mistral7 embeddings, the claim is there isn't any statistical difference between the two
# hypothesis,

    # is having more information better than less information in classification
#hypothesis 1:
    # having more information is better than less information
#hypotheisis 2:
    # are TFI-DF embeddings better than Mistral7 embeddings
# After running experiment s